In [4]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
from selenium import webdriver
import re
import pandas as pd
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
from tqdm import tqdm

In [5]:
driver = webdriver.Chrome()
driver.get('https://guide.michelin.com/fr/fr')
time.sleep(1)
driver.find_element('id', 'didomi-notice-agree-button').click() # accept cookies

In [6]:
div = driver.find_element('class name', "tabs-container__content--restaurant")
div.find_element('xpath', 'form/div/div/div').click()

In [7]:
input_element = driver.find_element('id', 'autocomplete-1-input')

In [8]:
try:
    input_element.click()
except:
    print('do tasks manually')

do tasks manually


normalement la dernière ligne plante pcq ya des protections anti scrap sur le site
à faire manuellement:
- Ctrl Maj I
- click droit sur l'input -> inspecter

In [9]:
input_element.send_keys('test')

In [10]:
def get_cities(country):
    req = requests.get('https://worldpopulationreview.com/countries/'+country)
    soup = BeautifulSoup(req.text, 'html.parser')
    cities = []
    for div in soup.find_all('div', {'class', 'lg:w-6/12 border-r px-2 py-4 last-of-type:border-r-0 md:px-4 lg:px-8'}):
        for th in div.find_all('th'):
            cities.append(th.text)
    return cities[2:]

In [14]:
def get_links(html:str):
    soup = BeautifulSoup(html, 'html.parser')
    data = []
    for i, section in enumerate(soup.find_all('section')):
        city_patterns = ['noindex']
        has_city_patterns = [False]
        region_patterns = ['region', 'state']
        has_region_patterns = [False, False]
        if i==1:
            break
        for a in section.find('ul').find_all('a'):
            e = a.get('href').split('/')
            region_url, city_url = e[3:5]
            stop = False

            for i, pattern in enumerate(city_patterns):
                if pattern in city_url:
                    if not has_city_patterns[i]: # add it once
                        if check_link(region_url, city_url):
                            data.append((region_url, city_url))
                            has_city_patterns[i] = True
                    stop = True
                    break
            if stop:
                continue
            for i, pattern in enumerate(region_patterns):
                if pattern in region_url:
                    if not has_region_patterns[i]: # add it once
                        if check_link(region_url, city_url):
                            data.append((region_url, city_url))
                            has_region_patterns[i] = True
                    stop = True
                    break
            if stop:
                continue
            if (region_url, city_url) not in data:
                if check_link(region_url, city_url):
                    data.append((region_url, city_url)) 
    return data

def check_link(region, city):
    req = requests.get(f'https://guide.michelin.com/fr/fr/{region}/{city}/restaurants?sort=distance')
    soup = BeautifulSoup(req.text, 'html.parser')
    div = soup.find('div', {'class', 'search-results__count'})
    return div.text.strip()!='Aucun résultat'

def search(text):
    input_element.send_keys(Keys.CONTROL,"a", Keys.DELETE)
    input_element.send_keys(text)
    try:
        html_content = driver.find_element("class name", "aa-PanelLayout")
        return get_links(html_content.get_attribute("outerHTML"))
    except NoSuchElementException:
        return []
    except StaleElementReferenceException: # when element takes time to load
        time.sleep(3)
        input_element.send_keys(Keys.CONTROL,"a", Keys.DELETE)
        input_element.send_keys(text)
        time.sleep(3)
        try:
            html_content = driver.find_element("class name", "aa-PanelLayout")
            return get_links(html_content.get_attribute("outerHTML"))
        except NoSuchElementException:
            return []

In [12]:
countries = []
req = requests.get('https://worldpopulationreview.com/countries')
soup = BeautifulSoup(req.text, 'html.parser')
for a in soup.find('table').find_all('a'):
    link = a.get('href')
    if 'flag' not in link:
        countries.append(link.split('/')[-1])

In [15]:
data = []
for i,country in tqdm(enumerate(countries)):
    cities = get_cities(country)
    for city in cities:
        for region_url, city_url in search(city):
            data.append({'country':country, 'city':city, 'region_url':region_url, 'city_url':city_url})
df = pd.DataFrame.from_dict(data)
df.to_csv('links_michelin.csv', index=False)
print('done')

234it [1:00:56, 15.63s/it]

done
